## Import

In [ ]:
import sys, copy, os
import numpy as np

sys.path.append('..')
from src.problem import load_problem
from src.evaluation import creer_matrice_ecarts
from src.optimpymoo.utlis import freq_from_index, interface_pymoo, identify_pareto, solution_freq, multiobjectivescores, drawhistory

from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.factory import get_algorithm, get_crossover, get_mutation, get_sampling
from pymoo.factory import get_termination
from pymoo.visualization.scatter import Scatter
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.algorithms.moo.nsga2 import NSGA2

%matplotlib inline

In [ ]:
# Chargement du fichier du problème :
problem_freq = load_problem('../tests/fapp05_0050.coo')

# interface problem <-> Pymoo ProblemClass
n_var, xl, xu = interface_pymoo(problem_freq)

## Particle Swarm Optimization (PSO)
- mono-objectif &rarr; pondérer les différents objectifs,
- optimisation continue &rarr; arrondir à l'entier le plus proche.

### Pour chaque agent :
<img src="optimpymoo/Capture d’écran 2021-02-11 à 15.20.36.png" style="width: 700px;"/>
<img src="optimpymoo/pso_light.png" style="width: 700px;"/>

### Globalement :
<img src="optimpymoo/ParticleSwarmArrowsAnimation.gif" style="width: 500px;"/>

### Etude :
- modifier la fonction mono-objectif (un ou plusieurs critères, poids relatifs) au niveau de *F["out"]*,
- modifier les paramètres de l'algorithme PSO (nombre d'agents, inertie, etc.).


In [ ]:
%matplotlib 
import matplotlib.pyplot as plt

def fonction_evaluation(problem, solution):
        eval_matrix = creer_matrice_ecarts(problem, solution)
        return 20 * eval_matrix.max() + eval_matrix.sum() + 50 * np.count_nonzero(eval_matrix)    

# Definition du problème :
class MyProblem1D(Problem):

    def __init__(self):
        super().__init__(n_var=n_var,
                         n_obj=1,
                         n_constr=0,
                         xl=xl,
                         xu=xu,
                         type_var=int,
                        elementwise_evaluation=True)

    def _evaluate(self, X, out, *args, **kwargs):
        solution = [freq_from_index(problem_freq, np.round(X[i]).astype(int)) for i in range(len(X))]
        out["F"] = np.array([fonction_evaluation(problem_freq, x) for x in solution])

problem_1D = MyProblem1D()

algorithm = PSO(pop_size=10, # Nb d'agents
                w=0.9, # terme d'inertie
                c1=2.0, # terme d'ancrage individuel
                C2=2.0, # terme d'ancrage collectif
                max_velocity_rate=0.025,
                adaptivebool=False,
                #sampling=LatinHypercubeSampling(),
                PerturbBest=False,
                seed=1)


result = minimize(problem_1D,
               algorithm,
               seed=1,
               save_history=True,
               verbose=False)

pred = solution_freq(problem_freq, result.X)
print("\nMeilleure solution trouvée :\n", pred)
print(f'\nCoûts associés aux 3 objectifs :\n', multiobjectivescores(problem_freq, pred))
drawhistory(problem_freq, result, f_eval = fonction_evaluation, algo="PSO")

## Non-dominated sorting genetic algorithm II (NSGA-II)
- multi-objectif,
- optimisation mixte (continue & discrète).

### Apport NSGA II :
<img src="optimpymoo/GA_speNSGA.jpeg" style="width: 400px;"/>

#### Rang des Fronts de Pareto & mesure de densité :
<img src="optimpymoo/Pareto-ordering-of-two-objectives.png" style="width: 400px;"/>

#### Apport NSGA II en détail :
<img src="optimpymoo/Operating-principle-of-NSGA2.png" style="width: 400px;"/>

### Etude :
- modifier les paramètres de l'algorithme NSGA2,
- comparer les résultats avec les autres algorithmes.

In [ ]:
# Definition du problème :
class MyProblem3D(Problem):

    def __init__(self):
        super().__init__(n_var=n_var,
                         n_obj=3,
                         n_constr=0,
                         xl=xl,
                         xu=xu,
                         type_var=int,
                        elementwise_evaluation=True)

    def _evaluate(self, X, out, *args, **kwargs):
        l_F = []
        for x in X:
            solution = freq_from_index(problem_freq, np.round(x).astype(int))
            eval_matrix = creer_matrice_ecarts(problem_freq, solution)
            f1 = eval_matrix.sum()
            f2 = eval_matrix.max()
            f3 = np.count_nonzero( eval_matrix )
            l_F.append([f1,f2,f3])
        out["F"] = np.array(l_F)


problem_3D = MyProblem3D()

In [ ]:
# Instantiation algo.
method = NSGA2(pop_size=500)


In [ ]:
%matplotlib

# Optimisation :
result = minimize(problem_3D,
               method,
               termination=('n_gen', 50),
               seed=1,
               save_history=True,
               verbose=False)

# Meilleure solution pour chaque objectif :
idx_best = result.F.argmin(axis=0)
pred=dict()
for ii,jj in enumerate(idx_best): 
    print(f'Meilleure solution pour critère n° {ii} :', multiobjectivescores(problem_freq, solution_freq(problem_freq, result.X[jj])))
drawhistory(problem_freq, result, f_eval = None, algo="NSGA2")